
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
 !pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y4", label="solidos")

/tmp/ipykernel_4859/178310800.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-24 18:03:46.141118: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 18:03:48.078604: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 18:03:48.078689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 18:03:48.364883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 18:03:48.971330: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 18:03:48.975863: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [10]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[70,35], [30,15], [35, 17]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[70, 35], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++


Elapsed time:  3.2965892880001775
Stopped at epoch:  67
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  5.976287234000665
Stopped at epoch:  127
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  2.5381252839997615
Stopped at epoch:  0
1/1 [==============================] - 0s 23ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.04291449999982433
Stopped at epoch:  0
1/1 [==============================] - 0s 19ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.03017782000006264
Stopped at epoch:  0
1/1 [==============================] - 0s 19ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.029467502999978024
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 7 ++++++++++++++++++
Elapsed time:  0.02995411299980333
Stopped at epoch:  0
1/1 [==============================] - 0s 22ms/ste

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_5_1_0,0.800278,0.512632,0.583874,0.556079,24.086517,45.752060,67.550781,56.010292
model_5_1_1,0.827861,0.462426,0.616864,0.556277,20.760000,50.465191,62.195393,55.985291


DataFrame salvo em ../results/metrics_5_1
+++++++++++ [5_2] | 1 ++++++++++++++++++
Elapsed time:  0.2323512920002031
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [5_2] | 2 ++++++++++++++++++
Elapsed time:  0.22083628099971975
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [5_2] | 3 ++++++++++++++++++
Elapsed time:  0.35110776700003044
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_2] | 4 ++++++++++++++++++
Elapsed time:  0.2501650049998716
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [5_2] | 5 ++++++++++++++++++
Elapsed time:  0.2465739819999726
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [5_2] | 6 ++++++++++++++++++
Elapsed time:  0.20437741600017034
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_2] | 7 ++++++++++++++++++
Elapsed time:  0.255752870

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_5_2_21,0.803383,0.827684,-0.980669,0.412153,23.712072,24.967529,102.768379,61.579624
model_5_2_20,0.808837,0.832607,-0.879917,0.439242,23.054321,24.254173,97.540802,58.741970
model_5_2_19,0.812899,0.837633,-0.800916,0.461335,22.564405,23.526018,93.441788,56.427608
model_5_2_18,0.818080,0.842245,-0.701125,0.487972,21.939581,22.857655,88.264038,53.637272
model_5_2_17,0.821783,0.846987,-0.624813,0.509232,21.493023,22.170708,84.304550,51.410122
model_5_2_16,0.826139,0.851203,-0.534618,0.533344,20.967655,21.559713,79.624710,48.884285
model_5_2_15,0.829271,0.855571,-0.463830,0.553041,20.589909,20.926857,75.951828,46.820957
model_5_2_0,0.831285,0.896734,0.137119,0.723256,20.347063,14.962542,44.771202,28.990164
model_5_2_14,0.832833,0.859450,-0.382908,0.574743,20.160360,20.364845,71.753151,44.547607
model_5_2_1,0.834196,0.892257,0.136781,0.719899,19.996073,15.611308,44.788708,29.341846


...

DataFrame salvo em ../results/metrics_7_6
+++++++++++ [7_7] | 1 ++++++++++++++++++
Elapsed time:  0.4603610700005447
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [7_7] | 2 ++++++++++++++++++
Elapsed time:  0.4738459280006282
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [7_7] | 3 ++++++++++++++++++
Elapsed time:  0.6441532419994473
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [7_7] | 4 ++++++++++++++++++
Elapsed time:  0.810859703999995
Stopped at epoch:  13
1/1 [==============================] - 0s 18ms/step
+++++++++++ [7_7] | 5 ++++++++++++++++++
Elapsed time:  0.34925251499953447
Stopped at epoch:  7
1/1 [==============================] - 0s 21ms/step
+++++++++++ [7_7] | 6 ++++++++++++++++++
Elapsed time:  0.8187831850000293
Stopped at epoch:  11
1/1 [==============================] - 0s 24ms/step
+++++++++++ [7_7] | 7 ++++++++++++++++++
Elapsed time:  0.5863129730

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_7_9_0,0.836872,0.428380,0.821032,0.691605,19.673277,52.956650,37.593979,45.727119
model_7_9_1,0.848220,0.464958,0.820239,0.703175,18.304758,49.568008,37.760544,44.011482
model_7_9_7,0.860321,0.527111,0.735950,0.667539,16.845343,43.809917,55.466412,49.295452
model_7_9_8,0.860928,0.501118,0.738086,0.660366,16.772079,46.217960,55.017609,50.359020
model_7_9_15,0.861459,0.378767,0.757011,0.632511,16.708050,57.552979,51.042149,54.489124
model_7_9_9,0.861719,0.484811,0.740943,0.656877,16.676704,47.728745,54.417454,50.876366
model_7_9_17,0.861944,0.373086,0.760594,0.633021,16.649616,58.079292,50.289700,54.413544
model_7_9_16,0.862218,0.376500,0.760346,0.633985,16.616587,57.763020,50.341621,54.270618
model_7_9_2,0.863752,0.518354,0.808957,0.713315,16.431492,44.621201,40.130554,42.507984
model_7_9_10,0.864863,0.512088,0.735505,0.662274,16.297512,45.201702,55.559753,50.076050


DataFrame salvo em ../results/metrics_7_9
Testando combinacao8: Hidden Size=[30, 15], regularizer=0.1, learning_rate=0.1
+++++++++++ [8_0] | 1 ++++++++++++++++++
Elapsed time:  0.876437480000277
Stopped at epoch:  12
1/1 [==============================] - 0s 18ms/step
+++++++++++ [8_0] | 2 ++++++++++++++++++
Elapsed time:  1.0621272630005478
Stopped at epoch:  12
1/1 [==============================] - 0s 22ms/step
+++++++++++ [8_0] | 3 ++++++++++++++++++
Elapsed time:  0.7216426519999004
Stopped at epoch:  10
1/1 [==============================] - 0s 21ms/step
+++++++++++ [8_0] | 4 ++++++++++++++++++
Elapsed time:  0.3865202329998283
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [8_0] | 5 ++++++++++++++++++
Elapsed time:  0.5617253889995482
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [8_0] | 6 ++++++++++++++++++
Elapsed time:  0.4207632369998464
Stopped at epoch:  7
1/1 [==============================] - 0s

2024-03-24 18:37:42.828785: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.7780289219999759
Stopped at epoch:  9
1/1 [==============================] - 0s 24ms/step
+++++++++++ [9_7] | 6 ++++++++++++++++++
Elapsed time:  1.2860009939995507
Stopped at epoch:  18
1/1 [==============================] - 0s 28ms/step
+++++++++++ [9_7] | 7 ++++++++++++++++++
Elapsed time:  0.6512919360002343
Stopped at epoch:  11
1/1 [==============================] - 0s 30ms/step
+++++++++++ [9_7] | 8 ++++++++++++++++++
Elapsed time:  0.6628424500004257
Stopped at epoch:  10
1/1 [==============================] - 0s 27ms/step
+++++++++++ [9_7] | 9 ++++++++++++++++++
Elapsed time:  1.3302306519999547
Stopped at epoch:  24
1/1 [==============================] - 0s 28ms/step
+++++++++++ [9_7] | 10 ++++++++++++++++++
Elapsed time:  0.6384614080006941
Stopped at epoch:  8
1/1 [==============================] - 0s 26ms/step
+++++++++++ [9_7] | 11 ++++++++++++++++++
Elapsed time:  1.4876558960004331
Stopped at epoch:  24
1/1 [==============================] - 0s 21ms/ste